In [ ]:
#OBSERVAÇÃO:
#Para executar corretamente este notebook, é preciso que os arquivos de entradas
#e saídas sejam carregados para o armazenamento da sessão

from numpy import array
from scipy.cluster.vq import vq, kmeans, whiten
from math import sqrt
from sklearn.cluster import KMeans

In [ ]:

def distancia(a,b):
  return sqrt( pow(a[0] - b[0],2) + pow(a[1] - b[1],2) )  

def centroide_proximo(ponto, clusters):
  
  indice_centroide = 0
  menor_distancia = distancia(ponto,clusters[0])
  for i in range(1, len(clusters)):
    dist = distancia(ponto,clusters[i])
    if(dist<menor_distancia):
      menor_distancia = dist
      indice_centroide = i
  
  return indice_centroide

def verifica_corretude(origem,nclusters):
  arq = open(origem,'r')
  i=0
  pontos = []
  for linha in arq:
    line = linha.split()
    if(i==0): 
      npontos = int(line[0])

    if(i==1):
      dimensao = int(line[0])

    if(i>1):
      ponto = []
      for j in range(dimensao):
        ponto.append(float(line[j]))
  
      pontos.append(ponto)
      ponto = []

    i+=1

  #print(npontos)
  #print(dimensao)
  #print(pontos)
  arq.close()

  pontos2 = array(pontos)
  #pontos2

  cent_iniciais = array([pontos2[i] for i in range(nclusters)])
  print("centroides iniciais: ", cent_iniciais)
  resultado = kmeans(pontos2,cent_iniciais,thresh=1e-06)

  #conferindo com a saida
  #primeiro verifica se os clusters são os mesmos
  #se sim, usa a lista de clusters_c pra verificar se
  #os pontos estão nos clusters certos

  saida = origem[:len(origem)-4] #pra tirar o .txt
  saida = saida+"c"+str(nclusters)+"_res.txt" #exemplo: p100d2c5.txt
  arq = open(saida,'r')

  clusters_py = resultado[0] #array de pontos [x,y]; resultado[1] é um outro
                             #valor que o kmeans retorna, chamado "distortion"

  clusters_c = []
  pontos_centroides_c = [] #uma lista com o valor v na casa i se o ponto i está 
                           #associado ao centroide de indice v na lista de
                           #clusters_c
  i=0
  for linha in arq:
    line = linha.split()
    if i<3: #não vou ler as primeiras 3 linhas
      i+=1
      continue

    if(i>=3 and i<3+nclusters):
      clusters_c.append([float(line[0]),float(line[1])])
  
    if(i>= 3+nclusters):
      pontos_centroides_c.append(int(line[0]))
    i+=1


  print("Clusters do C: ", clusters_c)
  print("Clusters do python: ", clusters_py)
  #print(pontos_centroides_c)

  arq.close()

  #vou comparar os clusters de c com os de python agora
  diferente=1
  for i in range(nclusters):
    x = clusters_py[i][0]
    #print(x)
    y = clusters_py[i][1]
    #print(y)
    diferente=1
    for j in range(nclusters):
      if(abs(clusters_c[j][0] - x)<1):
        if(abs(clusters_c[j][1] - y)<1):
          diferente = 0
          break

    j=0
    if(diferente ==1): #se o diferente ainda é igual a 1 é porque o 
                        #ponto é diferente de todos os outros
      print("os centroides são diferentes")
      #break
      return
    

  #se chegou aqui é porque os clusters são iguais
  pontos_centroides_py =[]
  for i in range(npontos):
    pontos_centroides_py.append(centroide_proximo(pontos[i], clusters_c))
    #por algum motivo, se botar clusters_py aqui da certo
    #mas da errado a partir de 10kc15


  if(pontos_centroides_py != pontos_centroides_c):
    print("os centroides estão certos, mas os pontos atribuídos a eles não")
    #como _py e _c estão usando a mesma lista de clusters e a mesma
    #lista de pontos -na mesma ordem-, deveria ser igual
    return
  
  #se chegou aqui está tudo certo
  print("Correto!")



def verifica_corretude_sklearn(origem,nclusters):
  arq = open(origem,'r')
  i=0
  pontos = []
  for linha in arq:
    line = linha.split()
    if(i==0): 
      npontos = int(line[0])

    if(i==1):
      dimensao = int(line[0])

    if(i>1):
      ponto = []
      for j in range(dimensao):
        ponto.append(float(line[j]))
  
      pontos.append(ponto)
      ponto = []

    i+=1

  #print(npontos)
  #print(dimensao)
  #print(pontos)
  arq.close()

  pontos2 = array(pontos)
  #pontos2

  cent_iniciais = array([pontos2[i] for i in range(nclusters)])
  #print("centroides iniciais: ", cent_iniciais)
  ##resultado = kmeans(pontos2,cent_iniciais,thresh=1e-06)
  
  kmeans = KMeans(n_clusters=nclusters,init=cent_iniciais,max_iter = 1000, tol=1e-06).fit(pontos2)
  clusters_py=kmeans.cluster_centers_

  #conferindo com a saida
  #primeiro verifica se os clusters são os mesmos
  #se sim, usa a lista de clusters_c pra verificar se
  #os pontos estão nos clusters certos

  saida = origem[:len(origem)-4] #pra tirar o .txt
  saida = saida+"c"+str(nclusters)+"_res.txt" #exemplo: p100d2c5.txt
  arq = open(saida,'r')

  #clusters_py = resultado[0] #array de pontos [x,y]; resultado[1] é um outro
                             #valor que o kmeans retorna, chamado "distortion"

  clusters_c = []
  pontos_centroides_c = [] #uma lista com o valor v na casa i se o ponto i está 
                           #associado ao centroide de indice v na lista de
                           #clusters_c
  i=0
  for linha in arq:
    line = linha.split()
    if i<3: #não vou ler as primeiras 3 linhas
      i+=1
      continue

    if(i>=3 and i<3+nclusters):
      clusters_c.append([float(line[0]),float(line[1])])
  
    if(i>= 3+nclusters):
      pontos_centroides_c.append(int(line[0]))
    i+=1


  #print("clusters do C: ", clusters_c)
  #print("clusters do python: ", clusters_py)
  #print(pontos_centroides_c)

  arq.close()

  #vou comparar os clusters de c com os de python agora
  diferente=1
  for i in range(nclusters):
    x = clusters_py[i][0]
    #print(x)
    y = clusters_py[i][1]
    #print(y)
    diferente=1
    for j in range(nclusters):
      if(abs(clusters_c[j][0] - x)<1e-06):
        if(abs(clusters_c[j][1] - y)<1e-06):
          diferente = 0
          break

    j=0
    if(diferente ==1): #se o diferente ainda é igual a 1 é porque o 
                        #ponto é diferente de todos os outros
      print("os centroides são diferentes")
      #break
      return
    

  #se chegou aqui é porque os clusters são iguais
  pontos_centroides_py =[]
  for i in range(npontos):
    pontos_centroides_py.append(centroide_proximo(pontos[i], clusters_c))
    #por algum motivo, se botar clusters_py aqui da certo
    #mas da errado a partir de 10kc15


  if(pontos_centroides_py != pontos_centroides_c):
    print("os centroides estão certos, mas os pontos atribuídos a eles não")
    #como _py e _c estão usando a mesma lista de clusters e a mesma
    #lista de pontos -na mesma ordem-, deveria ser igual
    return
  
  #se chegou aqui está tudo certo
  print("Correto!")

In [ ]:
origem = 'p100d2t1.txt'
verifica_corretude(origem,5)

centroides iniciais:  [[482. 227.]
 [323. 792.]
 [945. 312.]
 [289. 545.]
 [658.  22.]]
Clusters do C:  [[296.944458, 160.333328], [176.294113, 798.882324], [706.400024, 772.75], [178.5625, 414.8125], [752.862061, 274.93103]]
Clusters do python:  [[296.94444444 160.33333333]
 [176.29411765 798.88235294]
 [706.4        772.75      ]
 [178.5625     414.8125    ]
 [752.86206897 274.93103448]]
Correto!


In [ ]:
verifica_corretude(origem,15)

centroides iniciais:  [[482. 227.]
 [323. 792.]
 [945. 312.]
 [289. 545.]
 [658.  22.]
 [135. 863.]
 [912. 328.]
 [817. 911.]
 [567. 782.]
 [130. 325.]
 [653. 883.]
 [181. 829.]
 [512. 528.]
 [ 52. 149.]
 [920. 120.]]
Clusters do C:  [[382.272736, 148.090912], [353.0, 814.200012], [951.0, 324.0], [313.833344, 428.5], [659.727295, 270.090912], [111.428574, 859.0], [864.142883, 540.857117], [851.5, 849.166687], [599.75, 711.75], [116.5, 369.0], [590.400024, 916.599976], [96.75, 659.125], [520.799988, 490.799988], [96.666664, 174.555557], [780.375, 112.0]]
Clusters do python:  [[382.27272727 148.09090909]
 [353.         814.2       ]
 [951.         324.        ]
 [313.83333333 428.5       ]
 [659.72727273 270.09090909]
 [111.42857143 859.        ]
 [864.14285714 540.85714286]
 [851.5        849.16666667]
 [599.75       711.75      ]
 [116.5        369.        ]
 [590.4        916.6       ]
 [ 96.75       659.125     ]
 [520.8        490.8       ]
 [ 96.66666667 174.55555556]
 [780.375    

In [ ]:
verifica_corretude(origem,30)

centroides iniciais:  [[482. 227.]
 [323. 792.]
 [945. 312.]
 [289. 545.]
 [658.  22.]
 [135. 863.]
 [912. 328.]
 [817. 911.]
 [567. 782.]
 [130. 325.]
 [653. 883.]
 [181. 829.]
 [512. 528.]
 [ 52. 149.]
 [920. 120.]
 [110. 383.]
 [870.  85.]
 [326. 414.]
 [  5. 827.]
 [961. 343.]
 [367. 252.]
 [ 25. 211.]
 [707. 196.]
 [600. 577.]
 [990. 829.]
 [374.  33.]
 [424. 126.]
 [775. 102.]
 [377. 143.]
 [287. 483.]]
Clusters do C:  [[464.75, 247.0], [353.0, 814.200012], [951.0, 324.0], [99.5, 604.25], [639.666687, 73.0], [118.800003, 871.400024], [845.0, 521.200012], [809.0, 895.0], [556.333313, 733.333313], [131.666672, 294.333344], [590.400024, 916.599976], [175.0, 761.5], [501.0, 469.25], [148.333328, 124.333336], [920.0, 120.0], [131.0, 410.0], [870.0, 85.0], [308.666656, 403.0], [53.0, 747.25], [994.0, 522.0], [374.0, 293.0], [46.799999, 194.199997], [700.727295, 269.636353], [665.0, 612.0], [878.0, 767.0], [347.666656, 30.0], [424.0, 126.0], [781.0, 85.0], [344.333344, 160.0], [288.0, 5

In [ ]:
origem = 'p100d2t4.txt'
verifica_corretude(origem,5)

centroides iniciais:  [[981. 777.]
 [977.  37.]
 [743. 441.]
 [880. 414.]
 [ 70. 107.]]
Clusters do C:  [[667.578918, 898.947388], [681.764709, 104.764709], [193.310349, 753.310364], [772.285706, 407.928558], [226.095245, 265.809509]]
Clusters do python:  [[667.57894737 898.94736842]
 [681.76470588 104.76470588]
 [193.31034483 753.31034483]
 [772.28571429 407.92857143]
 [226.0952381  265.80952381]]
Correto!


In [ ]:
verifica_corretude(origem,15)

centroides iniciais:  [[981. 777.]
 [977.  37.]
 [743. 441.]
 [880. 414.]
 [ 70. 107.]
 [782.  80.]
 [576. 916.]
 [171. 471.]
 [ 26. 816.]
 [462. 709.]
 [258. 978.]
 [706. 265.]
 [837. 468.]
 [ 43.  80.]
 [170. 985.]]
Clusters do C:  [[882.571411, 836.142883], [937.333313, 60.333332], [682.142883, 405.571442], [859.400024, 348.200012], [289.0, 171.600006], [736.714294, 73.714287], [579.090881, 940.272705], [209.375, 428.125], [51.799999, 679.099976], [402.100006, 653.599976], [275.5, 891.833313], [503.777771, 190.555557], [837.0, 468.0], [73.5, 119.0], [145.399994, 933.799988]]
Clusters do python:  [[882.57142857 836.14285714]
 [937.33333333  60.33333333]
 [682.14285714 405.57142857]
 [859.4        348.2       ]
 [289.         171.6       ]
 [736.71428571  73.71428571]
 [579.09090909 940.27272727]
 [209.375      428.125     ]
 [ 51.8        679.1       ]
 [402.1        653.6       ]
 [275.5        891.83333333]
 [503.77777778 190.55555556]
 [837.         468.        ]
 [ 73.5        11

In [ ]:
verifica_corretude(origem,30)

centroides iniciais:  [[981. 777.]
 [977.  37.]
 [743. 441.]
 [880. 414.]
 [ 70. 107.]
 [782.  80.]
 [576. 916.]
 [171. 471.]
 [ 26. 816.]
 [462. 709.]
 [258. 978.]
 [706. 265.]
 [837. 468.]
 [ 43.  80.]
 [170. 985.]
 [288. 821.]
 [686. 906.]
 [200. 102.]
 [ 89.  46.]
 [578. 983.]
 [153. 341.]
 [ 47. 699.]
 [518. 119.]
 [771. 863.]
 [581.  24.]
 [ 84. 425.]
 [481. 991.]
 [522. 449.]
 [935. 136.]
 [ 16. 160.]]
Clusters do C:  [[948.666687, 717.0], [938.5, 22.5], [712.5, 450.5], [880.5, 351.5], [70.0, 107.0], [759.400024, 59.599998], [596.0, 880.666687], [222.25, 518.0], [25.75, 821.75], [458.0, 687.0], [269.666656, 954.666687], [739.25, 279.25], [837.0, 468.0], [43.0, 80.0], [171.0, 951.75], [277.399994, 768.400024], [669.666687, 933.0], [331.0, 171.0], [134.0, 78.0], [556.0, 982.0], [170.5, 297.0], [65.428574, 623.714294], [502.285706, 182.714279], [833.0, 925.5], [564.5, 54.0], [84.0, 425.0], [487.0, 979.333313], [420.799988, 415.600006], [935.0, 136.0], [16.0, 160.0]]
Clusters do pyt

In [ ]:
origem = 'p10kd2t1.txt'
verifica_corretude(origem,5)

centroides iniciais:  [[443. 990.]
 [ 34.  65.]
 [697. 215.]
 [873. 621.]
 [277. 226.]]
Clusters do C:  [[263.207245, 831.94989], [240.618835, 158.790024], [749.031494, 238.635178], [765.932861, 737.881592], [274.220428, 492.300934]]
Clusters do python:  [[263.20724234 831.94986072]
 [240.46087492 158.77510783]
 [748.92572388 238.61183382]
 [765.93286072 737.88156797]
 [274.22043309 492.30094392]]
Correto!


In [ ]:
verifica_corretude(origem,15)

centroides iniciais:  [[443. 990.]
 [ 34.  65.]
 [697. 215.]
 [873. 621.]
 [277. 226.]
 [851. 743.]
 [841. 386.]
 [910. 530.]
 [761.  89.]
 [542. 179.]
 [443. 778.]
 [369. 177.]
 [501. 415.]
 [929. 125.]
 [278. 652.]]
Clusters do C:  [[156.606323, 885.232788], [124.041031, 123.43161], [624.72583, 359.72583], [625.309265, 629.92511], [121.497643, 377.285706], [818.277771, 881.608582], [866.5, 400.250793], [883.754395, 646.677368], [619.78186, 122.554535], [377.033691, 138.632462], [479.18927, 881.181213], [373.010956, 410.194061], [358.513763, 668.145264], [878.759949, 132.680435], [117.157051, 631.69873]]
Clusters do python:  [[162.19852941 887.96176471]
 [126.46704871 129.73782235]
 [603.55502392 371.79425837]
 [640.84797768 638.31799163]
 [112.49172185 397.19205298]
 [818.26996198 882.41191381]
 [856.82701062 394.50986343]
 [892.33668342 644.32328308]
 [630.34769231 128.98769231]
 [389.47909968 125.6977492 ]
 [472.44201031 876.83634021]
 [350.49918434 374.55791191]
 [368.90322581 634

In [ ]:
verifica_corretude(origem,30)

centroides iniciais:  [[443. 990.]
 [ 34.  65.]
 [697. 215.]
 [873. 621.]
 [277. 226.]
 [851. 743.]
 [841. 386.]
 [910. 530.]
 [761.  89.]
 [542. 179.]
 [443. 778.]
 [369. 177.]
 [501. 415.]
 [929. 125.]
 [278. 652.]
 [234. 380.]
 [200. 699.]
 [196. 418.]
 [660. 378.]
 [954. 498.]
 [806. 402.]
 [876. 724.]
 [514. 561.]
 [211.  61.]
 [152. 130.]
 [387. 749.]
 [268. 196.]
 [  6. 357.]
 [753. 296.]
 [ 62. 879.]]
Clusters do C:  [[362.775208, 920.87854], [82.13308, 76.038025], [580.707764, 221.361038], [727.293152, 726.923279], [412.559509, 334.491058], [747.224243, 912.548462], [915.506042, 264.272736], [909.504089, 668.612488], [747.904602, 121.347694], [590.644165, 67.962547], [562.577271, 888.457703], [417.844391, 108.438042], [483.395355, 542.700562], [908.937317, 80.993401], [298.157562, 613.039368], [264.490509, 451.93988], [88.933525, 731.528931], [96.253822, 555.749207], [580.260864, 393.75], [908.122803, 475.084808], [711.331665, 549.039307], [915.724976, 879.219421], [503.092499

In [ ]:
origem = 'p10kd2t6.txt'
verifica_corretude(origem,5)

centroides iniciais:  [[485. 879.]
 [892. 208.]
 [714. 494.]
 [567. 335.]
 [551. 837.]]
Clusters do C:  [[226.909012, 745.532288], [754.989868, 161.723145], [683.983337, 495.125793], [244.667633, 238.043732], [747.113953, 827.051453]]
Clusters do python:  [[226.90901361 745.53231293]
 [754.9898568  161.72315036]
 [683.98333333 495.12580645]
 [244.66763848 238.04373178]
 [747.11396844 827.05143191]]
Correto!


In [ ]:
verifica_corretude(origem,15)

centroides iniciais:  [[485. 879.]
 [892. 208.]
 [714. 494.]
 [567. 335.]
 [551. 837.]
 [928. 316.]
 [734. 112.]
 [496. 550.]
 [715. 241.]
 [342.  88.]
 [773. 654.]
 [954.  63.]
 [732. 339.]
 [796. 701.]
 [115. 514.]]
Clusters do C:  [[151.468384, 887.484192], [672.236145, 351.832245], [630.563965, 631.741394], [151.338242, 391.602936], [489.393677, 883.452698], [885.241455, 425.875], [370.392212, 139.509094], [349.76889, 658.548523], [617.568176, 111.722404], [117.755486, 137.376175], [881.033752, 679.745361], [876.630676, 131.357147], [433.062927, 410.005585], [814.436523, 899.153259], [114.557533, 646.076172]]
Clusters do python:  [[151.4683908  887.4841954 ]
 [672.23615635 351.83224756]
 [630.56396149 631.74140303]
 [151.53404539 391.50333778]
 [489.39368999 883.4526749 ]
 [885.24143836 425.875     ]
 [370.39220779 139.50909091]
 [349.76887519 658.54853621]
 [617.56818182 111.7224026 ]
 [117.75548589 137.37617555]
 [881.03372681 679.74536256]
 [876.63069909 131.35714286]
 [433.2521

In [ ]:
verifica_corretude(origem,30)

centroides iniciais:  [[485. 879.]
 [892. 208.]
 [714. 494.]
 [567. 335.]
 [551. 837.]
 [928. 316.]
 [734. 112.]
 [496. 550.]
 [715. 241.]
 [342.  88.]
 [773. 654.]
 [954.  63.]
 [732. 339.]
 [796. 701.]
 [115. 514.]
 [478. 246.]
 [463. 368.]
 [311. 201.]
 [385. 393.]
 [714. 545.]
 [ 25. 151.]
 [406. 693.]
 [347. 975.]
 [527. 773.]
 [775. 629.]
 [181. 638.]
 [107. 322.]
 [979.  87.]
 [652. 882.]
 [955. 701.]]
Clusters do C:  [[307.693237, 909.625793], [888.260315, 235.161911], [743.008789, 547.412292], [577.929871, 397.091461], [523.858459, 916.667664], [918.713867, 436.867706], [510.286926, 72.82386], [391.972015, 574.206299], [642.741943, 226.919357], [298.504028, 85.948784], [606.511963, 734.56311], [726.05542, 80.483963], [759.504883, 358.744324], [779.442017, 733.12854], [87.608833, 567.933777], [436.122955, 229.0], [407.941711, 403.06134], [234.017288, 266.475494], [230.239365, 471.63031], [558.599976, 575.578125], [93.720695, 110.72818], [247.032089, 692.708557], [108.562325, 91

In [ ]:
origem = 'p1Md2t1.txt'
verifica_corretude(origem,5)

centroides iniciais:  [[626. 309.]
 [387. 123.]
 [489. 503.]
 [330. 534.]
 [654. 614.]]
Clusters do C:  [[775.848694, 218.71019], [219.326828, 219.475586], [498.130585, 496.106964], [219.439362, 775.655884], [776.044189, 775.978271]]
Clusters do python:  [[776.08393925 218.67832675]
 [218.93324422 219.15629088]
 [497.442721   494.51048565]
 [219.63886325 775.61553829]
 [775.67737697 775.70093965]]
os centroides são diferentes


In [ ]:
verifica_corretude(origem,15)

centroides iniciais:  [[626. 309.]
 [387. 123.]
 [489. 503.]
 [330. 534.]
 [654. 614.]
 [568. 210.]
 [434. 708.]
 [487. 424.]
 [241.  93.]
 [759. 841.]
 [119. 974.]
 [232. 779.]
 [ 68. 114.]
 [560. 259.]
 [ 60. 335.]]
Clusters do C:  [[867.611084, 389.603729], [622.989319, 114.675529], [625.413208, 634.003662], [359.21228, 707.994385], [880.646423, 640.562805], [873.587952, 131.423645], [519.947083, 893.070129], [373.28244, 443.959778], [372.769043, 149.366806], [836.105225, 882.028931], [157.286301, 886.890747], [117.410583, 622.198547], [124.499054, 117.217796], [613.099915, 361.837646], [129.149338, 361.568024]]
Clusters do python:  [[867.67776519 386.63249189]
 [623.63253805 116.53323805]
 [625.07988423 637.36061704]
 [358.83563418 707.7674617 ]
 [880.13191941 637.70609348]
 [873.72275236 130.16783609]
 [519.00107624 893.86645918]
 [372.86253779 443.93540549]
 [373.43568208 149.3596525 ]
 [836.64074513 881.46756593]
 [157.02021852 886.76846844]
 [117.28253249 621.98864176]
 [124.70

In [ ]:
verifica_corretude(origem,30)

centroides iniciais:  [[626. 309.]
 [387. 123.]
 [489. 503.]
 [330. 534.]
 [654. 614.]
 [568. 210.]
 [434. 708.]
 [487. 424.]
 [241.  93.]
 [759. 841.]
 [119. 974.]
 [232. 779.]
 [ 68. 114.]
 [560. 259.]
 [ 60. 335.]
 [447. 390.]
 [674. 177.]
 [273. 787.]
 [478. 861.]
 [  3. 492.]
 [132. 361.]
 [626. 621.]
 [223. 538.]
 [425. 379.]
 [ 37. 869.]
 [  1. 497.]
 [853. 938.]
 [494. 582.]
 [463. 902.]
 [120. 219.]]
Clusters do C:  [[893.050964, 282.14621], [539.725708, 84.319801], [598.556885, 585.986023], [414.460724, 573.607544], [920.279724, 484.900635], [736.711548, 90.327705], [480.063232, 748.053284], [733.58313, 417.176788], [330.364075, 79.952217], [911.90564, 707.757385], [106.671043, 914.772156], [282.504944, 740.243103], [110.788124, 76.49881], [663.039124, 251.677032], [75.354279, 238.301056], [533.044128, 409.655426], [913.728943, 95.005684], [318.385071, 913.827271], [718.661316, 920.004272], [109.981323, 401.944275], [328.73468, 401.404449], [776.203125, 594.692078], [235.4279

In [ ]:
origem = 'p1Md2t6.txt'
verifica_corretude(origem,5)

centroides iniciais:  [[721. 100.]
 [160.  63.]
 [700. 307.]
 [287. 947.]
 [738. 160.]]
Clusters do C:  [[499.066681, 285.115326], [164.340744, 248.483261], [750.328674, 757.592346], [246.022568, 757.789001], [833.101135, 249.131927]]
Clusters do python:  [[499.06671427 285.11531512]
 [164.34075522 248.48327048]
 [750.32897312 757.59192321]
 [246.02256026 757.788508  ]
 [833.10022744 249.13192832]]
Correto!


In [ ]:
verifica_corretude(origem,15)

centroides iniciais:  [[721. 100.]
 [160.  63.]
 [700. 307.]
 [287. 947.]
 [738. 160.]
 [279. 487.]
 [869. 799.]
 [468. 708.]
 [299. 369.]
 [ 82. 928.]
 [489. 169.]
 [678. 826.]
 [570. 139.]
 [916. 302.]
 [712. 678.]]
Clusters do C:  [[882.216797, 124.998344], [112.079659, 156.969803], [555.2229, 374.244141], [357.736908, 881.21167], [847.475708, 372.486511], [105.506592, 517.83667], [867.066589, 873.051575], [362.833374, 625.467224], [290.294006, 360.021454], [116.705002, 835.28186], [375.00473, 118.821983], [608.302429, 868.772156], [636.457825, 128.41774], [882.795105, 621.161438], [636.00116, 614.253113]]
Clusters do python:  [[881.93952385 124.55792595]
 [111.95587074 157.20642269]
 [555.44683358 374.99989816]
 [356.84153395 881.33965297]
 [847.35689485 371.50880734]
 [105.8374786  518.17323078]
 [866.80968354 872.56751673]
 [363.79282857 625.76585935]
 [290.48345467 360.22949166]
 [116.51521993 835.14085662]
 [374.5788383  118.89835131]
 [607.48102431 869.27783153]
 [635.73950092

In [ ]:
verifica_corretude(origem,30)

centroides iniciais:  [[721. 100.]
 [160.  63.]
 [700. 307.]
 [287. 947.]
 [738. 160.]
 [279. 487.]
 [869. 799.]
 [468. 708.]
 [299. 369.]
 [ 82. 928.]
 [489. 169.]
 [678. 826.]
 [570. 139.]
 [916. 302.]
 [712. 678.]
 [170. 763.]
 [690. 367.]
 [925. 148.]
 [558. 400.]
 [723. 132.]
 [849. 889.]
 [217. 133.]
 [653. 606.]
 [ 78. 906.]
 [770.  40.]
 [343. 227.]
 [145. 177.]
 [388. 905.]
 [212.   2.]
 [963. 403.]]
Clusters do C:  [[579.890381, 88.453484], [78.232483, 105.060654], [582.622437, 432.652466], [267.185547, 906.987976], [762.615662, 415.06665], [78.093628, 529.830261], [912.587341, 691.276672], [351.538483, 751.217041], [215.566574, 421.069031], [86.595261, 908.114624], [488.070435, 263.785034], [683.85791, 921.738892], [405.037292, 91.673584], [904.32312, 288.916534], [755.996765, 763.833801], [267.973419, 598.432861], [704.837402, 591.778625], [919.44397, 97.136604], [479.537476, 595.348633], [688.737976, 254.639572], [902.394226, 902.22998], [282.054688, 252.645218], [565.2559